# Lets save our data as a huggingface dataset, so it's quick to reuse



In [1]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


'4.30.1'

In [3]:
from src.prompts.format import format_guard_prompt, format_multishot
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
from src.datasets.batch import get_unique_config_hash, ds_params2fname
from src.datasets.hs import get_choices_as_tokens, default_class2choices, choice2ids, scores2choice_probs

# Params

In [4]:
# Params
BATCH_SIZE = 10  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True
# dataset_n = 200

# generation config
dataset_params = dict(
    model_repo="HuggingFaceH4/starchat-beta",
    dataset_name = "amazon_polarity",
    N = 1200, # 8000  # 4000 in 4 hours
    N_SHOTS = 3,
    prompt_fmt=format_guard_prompt,
    choices=default_class2choices,
)




# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large one might be best for lying.

In [5]:
model, tokenizer = load_model(model_repo=dataset_params['model_repo'])

changing pad_token_id from None to 0
changing padding_side from right to left
changing truncation_side from right to left



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0'), PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GPTBigCodeConfig {
  "_name_or_path": "HuggingFaceH4/starchat-beta",
  "activation_function": "gelu",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 6144,
  "n_head": 48,
  "n_inner": 24576,
  "n_layer": 40,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "resid_pdrop": 0.1,


# Load Dataset

In [6]:
dataset = load_dataset(dataset_params['dataset_name'])
dataset


Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})

## Format prompts

The prompt is the thing we most often have to change and debug. So we do it explicitly here.

We do it as transforms on a huggingface dataset.


In [7]:
def add_text_col_to_imbd_ds(r):
    return {"text": f"Review Title: {r['title']}\nReview Content: {r['content']}"}

def random_example(example_prompts):
    """gets a random example from shuffled train"""
    ex = dict(content=("blah blah " * 302))
    while (
        len(tokenizer(ex["content"]).input_ids) > 150
    ):  # 95% are below this 250, 70% below 150
        i = np.random.randint(example_prompts.num_rows)
        ex = example_prompts[i]
    return ex

example_prompts = dataset["train"].select(range(200)).map(add_text_col_to_imbd_ds)
get_random_example = functools.partial(random_example, example_prompts=example_prompts)


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-14b8893df38b4af0.arrow


In [8]:
def to_lie(label: int, lie: bool):
    if label == "":
        return ""
    else:
        return label ^ lie

def label_to_choice(label: bool) -> str:
    return ["False", "True"][label]


def format_imdb_multishot(input: str, response: str = "", lie: bool = False, fmt_prompt=dataset_params['prompt_fmt'], n_shots=dataset_params['N_SHOTS']) -> str:
    texts, responses = [], []
    for n in range(n_shots - 1):
        row = get_random_example()
        texts.append(row["text"])
        responses.append(label_to_choice(to_lie(row["label"], lie)))
    texts.append(input)

    if isinstance(response, int):
        response = label_to_choice(to_lie(response, lie))
    responses.append(response)
    return format_multishot(texts, responses, fmt_prompt=fmt_prompt)


In [32]:
lie = True
ds = (
    dataset["test"]
    .select(range(dataset_params["N"]))
    .map(add_text_col_to_imbd_ds)
    .map(lambda ex: {"prompt": format_imdb_multishot(ex["text"], lie=True), "lie": lie})
    .map(
        lambda ex: tokenizer(
            ex["prompt"], padding="max_length", max_length=600, truncation=True, add_special_tokens=True,
            # return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True,
    )
    .map(
        lambda r: {"prompt_truncated": tokenizer.batch_decode(r["input_ids"])},
        batched=True,
    )
)
ds


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'title', 'content', 'text', 'prompt', 'lie', 'input_ids', 'attention_mask', 'prompt_truncated'],
    num_rows: 1200
})

## Save as Huggingface Dataset

In [33]:
config_hash, info_kwargs = get_unique_config_hash(
    format_imdb_multishot, model, tokenizer, ds, dataset_params['N']
)
dataset_name = ds_params2fname(dataset_params) + config_hash
f = f"../.ds/{dataset_name}"
f

'../.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N1200_3shots_8a53f8'

## Generate dataset

In [34]:
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds,
    n=dataset_params['N'],
    batch_size=BATCH_SIZE,
)
gen_kwargs

{'model': GPTBigCodeForCausalLM(
   (transformer): GPTBigCodeModel(
     (wte): Embedding(49156, 6144)
     (wpe): Embedding(8192, 6144)
     (drop): Dropout(p=0.1, inplace=False)
     (h): ModuleList(
       (0-39): 40 x GPTBigCodeBlock(
         (ln_1): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (attn): GPTBigCodeAttention(
           (c_attn): Linear4bit(in_features=6144, out_features=6400, bias=True)
           (c_proj): Linear4bit(in_features=6144, out_features=6144, bias=True)
           (attn_dropout): Dropout(p=0.1, inplace=False)
           (resid_dropout): Dropout(p=0.1, inplace=False)
         )
         (ln_2): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (mlp): GPTBigCodeMLP(
           (c_fc): Linear4bit(in_features=6144, out_features=24576, bias=True)
           (c_proj): Linear4bit(in_features=24576, out_features=6144, bias=True)
           (act): GELUActivation()
           (dropout): Dropout(p=0.1, inplace=False)
         )
    

In [35]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=f"kwargs={info_kwargs} dataset_params={dataset_params}",
        config_name=f,
               
    ),
    gen_kwargs=gen_kwargs,
    # config_kwargs={},
).with_format("numpy")
ds1

Generating train split: 0 examples [00:00, ? examples/s]

get hidden states:   0%|          | 0/120 [00:00<?, ?it/s]

## Add labels

In [ ]:
class2_ids = choice2ids(tokenizer, dataset_params['choices'])
add_txt_ans = lambda r: {'txt_ans': tokenizer.decode(r['scores1'].argmax(-1))}

ds3 = (
    ds1
    .map(lambda r: scores2choice_probs(r, class2_ids))
    .map(add_txt_ans)
)
ds3

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 add_txt_ans = lambda r: {'txt_ans': tokenizer.decode(r['scores1'].argmax(-1))}              │
│    3                                                                                             │
│    4 ds3 = (                                                                                     │
│ ❱  5 │   ds1                                                                                     │
│    6 │   .map(lambda r: scores2choice_probs(r, class2_ids))                                      │
│    7 │   .map(add_txt_ans)                                                                       │
│    8 )                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds1' is not defined

## Save to disk

In [ ]:
ds3.save_to_disk(f)
f

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ds3.save_to_disk(f)                                                                          │
│   2 f                                                                                            │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds3' is not defined

# QC

In [ ]:
ds4 = load_from_disk(f)
ds4

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ds4 = load_from_disk(f)                                                                      │
│   2 ds4                                                                                          │
│   3                                                                                              │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/datasets/load.py:1886 in           │
│ load_from_disk                                                                                   │
│                                                                                                  │
│   1883 │   │   path_join = os.path.join                                                          │
│   1884 │                                                                                         │
│   1885 │   if not fs.exists(dest_dataset_path):                                                  │
│ ❱ 1886 │   │   raise FileNotFoundError(f"Directory {dataset_path} not found")                    │
│   1887 │   if fs.isfile(path_join(dest_dataset_path, config.DATASET_INFO_FILENAME)) and fs.isfi  │
│   1888 │   │   path_join(dest_dataset_path, config.DATASET_STATE_JSON_FILENAME)                  │
│   1889 │   ):                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: Directory ../.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N1200_3shots_0328e7 
not found

In [ ]:

# QC, check which answers are most common
common_answers = pd.Series(ds4['txt_ans']).value_counts()
display('Remember it should be binary. Found common LLM answers:', common_answers)

# list unexpected answers
class2choices = dataset_params['choices']
current_choices = set(class2choices[0]+class2choices[1])
unexpected_answers = set(common_answers.head(10).index)-current_choices
if len(unexpected_answers):
    logger.warning(f'found unexpected answers: {unexpected_answers}. You may want to add them to class2choices')
    
mean_prob = ds4['choice_probs1'].sum(-1).mean()
print('mean_prob', mean_prob)
assert ds4['choice_probs1'].sum(-1).mean()>0.4, f"""
Our choices should cover most common answers. But they accounted for a mean probability of {mean_prob:2.2%} (should be >40%). 

To fix this you might want to improve your prompt or add to your choices
"""

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # QC, check which answers are most common                                                   │
│ ❱  2 common_answers = pd.Series(ds4['txt_ans']).value_counts()                                   │
│    3 display('Remember it should be binary. Found common LLM answers:', common_answers)          │
│    4                                                                                             │
│    5 # list unexpected answers                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds4' is not defined

In [ ]:
df = ds2df(ds4)
df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df = ds2df(ds4)                                                                              │
│   2 df                                                                                           │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds4' is not defined

In [ ]:
# QC check accuracy
# it should manage to lie some of the time when asked to lie. Many models wont lie unless very explicitly asked to, but we don't want to do that, we want to leave some ambiguity in the prompt

d = df.query('lie==True')
acc = (d.desired_ans==d.llm_ans).mean()
print(f"when the model tries to lie... we get this acc {acc:2.2f}")
assert acc>0.1, f"should be acc>0.1 but is acc={acc}"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 # QC check accuracy                                                                          │
│   2 # it should manage to lie some of the time when asked to lie. Many models wont lie unles     │
│   3                                                                                              │
│ ❱ 4 d = df.query('lie==True')                                                                    │
│   5 acc = (d.desired_ans==d.llm_ans).mean()                                                      │
│   6 print(f"when the model tries to lie... we get this acc {acc:2.2f}")                          │
│   7 assert acc>0.1, f"should be acc>0.1 but is acc={acc}"                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df' is not defined

In [ ]:
# QC by viewing a row
r = ds4[0]
print(r['prompt_truncated'][0])
print(r['txt_ans'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # QC by viewing a row                                                                        │
│ ❱ 2 r = ds4[0]                                                                                   │
│   3 print(r['prompt_truncated'][0])                                                              │
│   4 print(r['txt_ans'])                                                                          │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds4' is not defined

# QC: generation

Let's a quick generation, so we can QC the output and sanity check that the model can actually do the task

In [ ]:
# r = ds[2]
# q = r["prompt_truncated"]

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )
# sequences = pipeline(
#     q.lstrip('<|endoftext|>'),
#     max_length=100,
#     do_sample=False,
#     return_full_text=False,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print("-" * 80)
#     print(q)
#     print("-" * 80)
#     print(f"`{seq['generated_text']}`")
#     print("-" * 80)
#     print("label", r['label'])
